In [1]:
# import necessary libraries

import pandas as pd
import numpy as np
import gensim
from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import math

In [2]:
# read csv file

file_path = r'C:\Users\User\Desktop\Recommender_System\games_of_all_time.csv'
df = pd.read_csv(file_path)
df

,game_name,meta_score,user_score,platform,description,url,developer,genre,type,rating
0,The Legend of Zelda: Ocarina of Time,99.0,91.0,['nintendo-64'],"As a young boy, Link is tricked by Ganondorf, ...",https://www.metacritic.com/game/nintendo-64/th...,Nintendo,"['Action Adventure', 'Fantasy']",singleplayer,E
1,Super Mario Galaxy,97.0,91.0,['wii'],[Metacritic's 2007 Wii Game of the Year] The u...,https://www.metacritic.com/game/wii/super-mari...,Nintendo,"['Action', 'Platformer', '3D']",singleplayer,E
2,Super Mario Galaxy 2,97.0,91.0,['wii'],"Super Mario Galaxy 2, the sequel to the galaxy...",https://www.metacritic.com/game/wii/super-mari...,Nintendo EAD Tokyo,"['Action', 'Platformer', '3D']",singleplayer,E
3,Metroid Prime,97.0,89.0,['gamecube'],Samus returns in a new mission to unravel the ...,https://www.metacritic.com/game/gamecube/metro...,Retro Studios,"['Action', 'Shooter', 'First-Person', 'Sci-Fi']",singleplayer,T
4,Super Mario Odyssey,97.0,89.0,['switch'],New Evolution of Mario Sandbox-Style Gameplay....,https://www.metacritic.com/game/switch/super-m...,Nintendo,"['Action', 'Platformer', '3D']",singleplayer,E10+
...,...,...,...,...,...,...,...,...,...,...
8826,kill.switch,74.0,80.5,"['xbox', 'playstation-2']","In a world on the brink of global conflict, yo...",https://www.metacritic.com/game/xbox/killswitch,Namco,"['Action', 'Shooter', 'Third-Person', 'Modern']",singleplayer,T
8827,nail'd,69.0,76.0,['pc'],nail’d is all about eschewing boring realism f...,https://www.metacritic.com/game/pc/naild,Techland,"['Driving', 'Racing', 'Arcade', 'Automobile']",singleplayer,E10+
8828,oOo: Ascension,70.0,73.0,['xbox-one'],oOo: Ascension is a lightning-paced test of re...,https://www.metacritic.com/game/xbox-one/ooo-a...,Kenny Creanor,"['Action', 'General', 'Arcade']",multiplayer,E
8829,uDraw Studio,71.0,71.0,['wii'],"Packaged with the uDraw GameTablet, uDraw Stud...",https://www.metacritic.com/game/wii/udraw-studio,THQ,"['Miscellaneous', 'General']",NaN,E


In [3]:
# show number of null values in each column

df.isna().sum()

game_name         0
meta_score        0
user_score        0
platform          0
description       0
url               0
developer        10
genre             4
type           2104
rating         1140
dtype: int64

In [4]:
# remove recordes with null values

df = df.dropna()
df.shape

(5933, 10)

In [5]:
# remove noisy data in numerical columns

numerical_features = ['meta_score','user_score']
for column in numerical_features:
    for element in df[column]:
        if isinstance(type(element),float) or element > 100 or element < 0 :
            df = df.drop(df.index[ df[column] == element])
df.shape

(5933, 10)

In [6]:
# remove noisy data in rating column and recorrecting value 'E10+'
# reindex the dataset

for element in df['rating']:
    if element == 'K-A' or element == 'RP':
        df = df.drop(df.index[df['rating'] == element])
    if element == 'E10+':
        df['rating'] = df['rating'].replace('E10+', 'E')
df.index = range(len(df.index))
df.shape
df

C:\Users\User\AppData\Local\Temp\ipykernel_17932\3705151528.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating'] = df['rating'].replace('E10+', 'E')


,game_name,meta_score,user_score,platform,description,url,developer,genre,type,rating
0,The Legend of Zelda: Ocarina of Time,99.0,91.0,['nintendo-64'],"As a young boy, Link is tricked by Ganondorf, ...",https://www.metacritic.com/game/nintendo-64/th...,Nintendo,"['Action Adventure', 'Fantasy']",singleplayer,E
1,Super Mario Galaxy,97.0,91.0,['wii'],[Metacritic's 2007 Wii Game of the Year] The u...,https://www.metacritic.com/game/wii/super-mari...,Nintendo,"['Action', 'Platformer', '3D']",singleplayer,E
2,Super Mario Galaxy 2,97.0,91.0,['wii'],"Super Mario Galaxy 2, the sequel to the galaxy...",https://www.metacritic.com/game/wii/super-mari...,Nintendo EAD Tokyo,"['Action', 'Platformer', '3D']",singleplayer,E
3,Metroid Prime,97.0,89.0,['gamecube'],Samus returns in a new mission to unravel the ...,https://www.metacritic.com/game/gamecube/metro...,Retro Studios,"['Action', 'Shooter', 'First-Person', 'Sci-Fi']",singleplayer,T
4,Super Mario Odyssey,97.0,89.0,['switch'],New Evolution of Mario Sandbox-Style Gameplay....,https://www.metacritic.com/game/switch/super-m...,Nintendo,"['Action', 'Platformer', '3D']",singleplayer,E
...,...,...,...,...,...,...,...,...,...,...
5916,inFamous,85.0,83.0,['playstation-3'],A massive explosion rips through six square bl...,https://www.metacritic.com/game/playstation-3/...,Sucker Punch,"['Action Adventure', 'Modern', 'Sci-Fi', 'Open...",singleplayer,T
5917,kill.switch,74.0,80.5,"['xbox', 'playstation-2']","In a world on the brink of global conflict, yo...",https://www.metacritic.com/game/xbox/killswitch,Namco,"['Action', 'Shooter', 'Third-Person', 'Modern']",singleplayer,T
5918,nail'd,69.0,76.0,['pc'],nail’d is all about eschewing boring realism f...,https://www.metacritic.com/game/pc/naild,Techland,"['Driving', 'Racing', 'Arcade', 'Automobile']",singleplayer,E
5919,oOo: Ascension,70.0,73.0,['xbox-one'],oOo: Ascension is a lightning-paced test of re...,https://www.metacritic.com/game/xbox-one/ooo-a...,Kenny Creanor,"['Action', 'General', 'Arcade']",multiplayer,E


In [7]:
# drop unnecessary features

df = df.drop(['url'], axis=1)

In [8]:
# remove unnecessary characters dataframe in categorical features

df_categorical_features = df.select_dtypes(include='object')
for column in df_categorical_features:
    df[column] = df[column].apply(lambda x: x.replace("[", ""))
    df[column] = df[column].apply(lambda x: x.replace("]", ""))
    df[column] = df[column].apply(lambda x: x.replace("'", ""))

In [9]:
# merge categorical columns 

df['Merge_Features'] = df[df.columns[3:]].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)

In [10]:
# Remove unnecessary characters in Merge_Features column

df['Merge_Features'] = df['Merge_Features'].apply(lambda x: x.replace(",", " "))
df['Merge_Features'] = df['Merge_Features'].apply(lambda x: x.replace(".", " "))
df['Merge_Features'] = df['Merge_Features'].apply(lambda x: x.replace("!", " "))
df['Merge_Features'] = df['Merge_Features'].apply(lambda x: x.replace("_", " "))

In [11]:
# lowercasing the string and split string values in Merge_Features column

df['Merge_Features'] = df['Merge_Features'].apply(lambda x: x.lower())
df['Merge_Features'] = df['Merge_Features'].apply(lambda x: x.split(' '))

In [12]:
# remove space character in splited values

df['Merge_Features'] = df['Merge_Features'].apply(lambda x: [i.replace(' ','') for i in x])

In [13]:
# deleting space elements in lists

for i in range(len(df['Merge_Features'])):
    for element in df['Merge_Features'][i]:
        if '' in df['Merge_Features'][i]:
            df['Merge_Features'][i].remove('')
df['Merge_Features'][1]

['wii',
 'metacritics',
 '2007',
 'wii',
 'game',
 'of',
 'the',
 'year',
 'the',
 'ultimate',
 'nintendo',
 'hero',
 'is',
 'taking',
 'the',
 'ultimate',
 'step',
 'out',
 'into',
 'space',
 'join',
 'mario',
 'as',
 'he',
 'ushers',
 'in',
 'a',
 'new',
 'era',
 'of',
 'video',
 'games',
 'defying',
 'gravity',
 'across',
 'all',
 'the',
 'planets',
 'in',
 'the',
 'galaxy',
 'when',
 'some',
 'creature',
 'escapes',
 'into',
 'space',
 'with',
 'princess',
 'peach',
 'mario',
 'gives',
 'chase',
 'exploring',
 'bizarre',
 'planets',
 'all',
 'across',
 'the',
 'galaxy',
 'mario',
 'peach',
 'and',
 'enemies',
 'new',
 'and',
 'old',
 'are',
 'here',
 'players',
 'run',
 'jump',
 'and',
 'battle',
 'enemies',
 'as',
 'they',
 'explore',
 'all',
 'the',
 'planets',
 'in',
 'the',
 'galaxy',
 'since',
 'this',
 'game',
 'makes',
 'full',
 'use',
 'of',
 'all',
 'the',
 'features',
 'of',
 'the',
 'wii',
 'remote',
 'players',
 'have',
 'to',
 'do',
 'all',
 'kinds',
 'of',
 'things',


In [14]:
# joining words to create text in Merge_Features column

df['Merge_Features'] = df['Merge_Features'].apply(lambda x: ' '.join(x))

In [15]:
# define function that remove useless words

def remove_useless_words(text):
    filtered_sentence = remove_stopwords(text)
    return filtered_sentence

In [16]:
# remove useless words from texts

df['main_text'] = df['Merge_Features'].apply(lambda x: remove_useless_words(x))

In [17]:
# define a function to Remove verb suffixes

def remove_suffix(text):
    ps = PorterStemmer()
    StemList = []
    for word in text.split():
        StemList.append(ps.stem(word))
    return ' '.join(StemList)

In [18]:
# remove verb suffixes from texts

df['main_text'] = df['main_text'].apply(lambda x: remove_suffix(x))

In [19]:
# COUNT VECTORIZER (or ONE HOT ENCODING)
# features are extracted

cv = CountVectorizer()
fit_main_text = cv.fit_transform(df['main_text'])
fit_main_text

<5921x21782 sparse matrix of type '<class 'numpy.int64'>'
	with 345640 stored elements in Compressed Sparse Row format>

In [20]:
# Defines a dataframe that represents numeric vectors

df_cv = pd.DataFrame(fit_main_text.toarray(), columns=cv.get_feature_names())
df_cv

c:\Users\User\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,00,000,007,0079,0083,01,02,03,04,05,...,zuul,zuxxez,zx,zxibit,zyre,â½,état,überassassin,сiti,ﬁnding
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5916,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5917,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5918,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5919,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
df

,game_name,meta_score,user_score,platform,description,developer,genre,type,rating,Merge_Features,main_text
0,The Legend of Zelda: Ocarina of Time,99.0,91.0,nintendo-64,"As a young boy, Link is tricked by Ganondorf, ...",Nintendo,"Action Adventure, Fantasy",singleplayer,E,nintendo-64 as a young boy link is tricked by ...,nintendo-64 young boy link trick ganondorf kin...
1,Super Mario Galaxy,97.0,91.0,wii,Metacritics 2007 Wii Game of the Year The ulti...,Nintendo,"Action, Platformer, 3D",singleplayer,E,wii metacritics 2007 wii game of the year the ...,wii metacrit 2007 wii game year ultim nintendo...
2,Super Mario Galaxy 2,97.0,91.0,wii,"Super Mario Galaxy 2, the sequel to the galaxy...",Nintendo EAD Tokyo,"Action, Platformer, 3D",singleplayer,E,wii super mario galaxy 2 the sequel to the gal...,wii super mario galaxi 2 sequel galaxy-hop ori...
3,Metroid Prime,97.0,89.0,gamecube,Samus returns in a new mission to unravel the ...,Retro Studios,"Action, Shooter, First-Person, Sci-Fi",singleplayer,T,gamecube samus returns in a new mission to unr...,gamecub samu return new mission unravel myster...
4,Super Mario Odyssey,97.0,89.0,switch,New Evolution of Mario Sandbox-Style Gameplay....,Nintendo,"Action, Platformer, 3D",singleplayer,E,switch new evolution of mario sandbox-style ga...,switch new evolut mario sandbox-styl gameplay ...
...,...,...,...,...,...,...,...,...,...,...,...
5916,inFamous,85.0,83.0,playstation-3,A massive explosion rips through six square bl...,Sucker Punch,"Action Adventure, Modern, Sci-Fi, Open-World",singleplayer,T,playstation-3 a massive explosion rips through...,playstation-3 massiv explos rip squar block em...
5917,kill.switch,74.0,80.5,"xbox, playstation-2","In a world on the brink of global conflict, yo...",Namco,"Action, Shooter, Third-Person, Modern",singleplayer,T,xbox playstation-2 in a world on the brink of ...,xbox playstation-2 world brink global conflict...
5918,naild,69.0,76.0,pc,nail’d is all about eschewing boring realism f...,Techland,"Driving, Racing, Arcade, Automobile",singleplayer,E,pc nail’d is all about eschewing boring realis...,pc nail’d eschew bore realism arcad fun it’ ra...
5919,oOo: Ascension,70.0,73.0,xbox-one,oOo: Ascension is a lightning-paced test of re...,Kenny Creanor,"Action, General, Arcade",multiplayer,E,xbox-one ooo: ascension is a lightning-paced t...,xbox-on ooo: ascens lightning-pac test reflex ...


In [22]:
# Define a function that measures the similarity between two games

def euclidean_distance(v1,v2):
    sum_distance = 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sum_distance += (v1[i] - v2[i])*(v1[i] - v2[i])
    return math.sqrt(sum_distance)

In [23]:
# First question about platform

while True:
    try:
        platform = input("What is your desired game platform? (If the game runs on more than one platform, separate them with (,).)")
    except ValueError:
        print("Sorry, I didn't understand that.")
        continue
    else:
        break

In [24]:
# Second Question about Game description

while True:
    try:
        description = input("Describe your desired game in a few sentences. ")
    except ValueError:
        print("Sorry, I didn't understand that.")
        continue
    else:
        break

In [25]:
# Third Question about developer

while True:
    try:
        developer = input("Who is the developer of your desired game? ")
    except ValueError:
        print("Sorry, I didn't understand that.")
        continue
    else:
        break

In [26]:
# Forth question about genres

while True:
    try:
        genres = input("What are your desired game genres? (If there is more than one genre, separate them with (,).)")
    except ValueError:
        print("Sorry, I didn't understand that.")
        continue
    else:
        break

In [27]:
# Fifth question about game type (singleplayer or multiplayer)

while True:
    try:
        type = input("What kind of game do you want? (singleplayer or multiplayer) ")
    except ValueError:
        print("Sorry, I didn't understand that.")
        continue

    if type != 'singleplayer' and type != 'multiplayer':
        print("Sorry, your response is wrong.")
        continue
    else:
        break

In [28]:
# Sixth question about game rating

while True:
    try:
        rating = input("What is your game rating? (E , T or M) ")
    except ValueError:
        print("Sorry, I didn't understand that.")
        continue

    substring = ['T','M','E']
    if rating not in substring:
        print("Sorry, your response is wrong.")
        continue
    else:
        break

In [29]:
# Create user dataframe

user_data = {'platform': str(platform), 'description': str(description), 'developer': str(developer), 'genres': str(genres), 'type': str(type), 'rating': str(rating),}
df_user = pd.DataFrame(user_data, index=[0])
df_user

,platform,description,developer,genres,type,rating
0,"xbox, playstation-4",a pirate who fights against other pirates. dis...,roo,"action, survival",multiplayer,T


In [30]:
# remove unnecessary characters dataframe in categorical features

df_user_categorical_features = df_user.select_dtypes(include='object')
for column in df_user_categorical_features:
    df_user[column] = df_user[column].apply(lambda x: x.replace("[", ""))
    df_user[column] = df_user[column].apply(lambda x: x.replace("]", ""))
    df_user[column] = df_user[column].apply(lambda x: x.replace("'", ""))

In [31]:
# merge categorical columns

df_user['Merge_user_data'] = df_user[df_user.columns].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)

In [32]:
# Remove unnecessary characters in Merge_user_data column

df_user['Merge_user_data'] = df_user['Merge_user_data'].apply(lambda x: x.replace(",", " "))
df_user['Merge_user_data'] = df_user['Merge_user_data'].apply(lambda x: x.replace("!", " "))
df_user['Merge_user_data'] = df_user['Merge_user_data'].apply(lambda x: x.replace(".", " "))
df_user['Merge_user_data'] = df_user['Merge_user_data'].apply(lambda x: x.replace("_", " "))

In [33]:
# lowercasing the string and split string values in Merge_user_data column

df_user['Merge_user_data'] = df_user['Merge_user_data'].apply(lambda x: x.lower())
df_user['Merge_user_data'] = df_user['Merge_user_data'].apply(lambda x: x.split(' '))

In [34]:
# remove space character in splited values

df_user['Merge_user_data'] = df_user['Merge_user_data'].apply(lambda x: [i.replace(' ','') for i in x])

In [35]:
# deleting space elements in lists

for element in df_user['Merge_user_data']:
    if element == '' :
        df_user['Merge_user_data'].remove(element)
df_user['Merge_user_data']

0    [xbox, , playstation-4, a, pirate, who, fights...
Name: Merge_user_data, dtype: object

In [36]:
# joining words to create text in Merge_user_data column

df_user['Merge_user_data'] = df_user['Merge_user_data'].apply(lambda x: ' '.join(x))
df_user['Merge_user_data']

0    xbox  playstation-4 a pirate who fights agains...
Name: Merge_user_data, dtype: object

In [37]:
# remove useless words from text

df_user['main_text_data'] = df_user['Merge_user_data'].apply(lambda x: remove_useless_words(x))
df_user['main_text_data']

0    xbox playstation-4 pirate fights pirates disco...
Name: main_text_data, dtype: object

In [38]:
# remove verb suffixes from text

df_user['main_text_data'] = df_user['main_text_data'].apply(lambda x: remove_suffix(x))
df_user['main_text_data']

0    xbox playstation-4 pirat fight pirat discov un...
Name: main_text_data, dtype: object

In [39]:
# Determine how many of the previous words are present in the new text

data_countvectorizer_dict = {}
keys = cv.get_feature_names()
values = []
for word in cv.get_feature_names():
    values.append(list(df_user['main_text_data'].str.split()).count(word))
for key in keys:
    for value in values:
        data_countvectorizer_dict[key] = value

c:\Users\User\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [40]:
# create dataframe for new vector

df_data_countvectorizer = pd.DataFrame(data_countvectorizer_dict, index=[0])
df_data_countvectorizer

,00,000,007,0079,0083,01,02,03,04,05,...,zuul,zuxxez,zx,zxibit,zyre,â½,état,überassassin,сiti,ﬁnding
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
# calculate euclidean distance between previous games and new game

euclidean_distance_games = []
for previous_game_index in range(len(df_cv)):
    euclidean_distance_games.append(euclidean_distance(np.array(df_cv.loc[previous_game_index]),np.array(df_data_countvectorizer.loc[0])))

In [ ]:
# Creating a dictionary that shows each of the previous games along with the euclidean distance with the new game

game_distance_dict = {}    
keys = df['game_name']
values = []  
for game_index in range(len(df['game_name'])):
    values.append(euclidean_distance_games[game_index])
for i in range(len(df['game_name'])):
    game_distance_dict[keys[i]] = values[i]

In [ ]:
# sorting the game distance dictionary

sorted_values = sorted(game_distance_dict.values()) 
sorted_game_distance_dict = {}

for i in sorted_values:
    for k in game_distance_dict.keys():
        if game_distance_dict[k] == i:
            sorted_game_distance_dict[k] = game_distance_dict[k]
print(sorted_game_distance_dict)

In [64]:
N = 10
d1= dict(list(sorted_game_distance_dict.items())[0: N]) 
print(list(d1.keys()))

['Flood of Light', 'Men in Black - The Series: Crashdown', 'Atelier Arland Series Deluxe Pack', 'N++', 'Aaero: Complete Edition', 'Light Fall', 'Vesta', 'Lost in Harmony', 'Haunted Dungeons: Hyakki Castle', 'Accel World vs. Sword Art Online: Millennium Twilight']
